In [2]:
import numpy as np
import pandas as pd

from asteroid_dataframe import load_ast_data, calc_ast_data, spline_ast_vec_df, calc_ast_dir
from asteroid_data import orbital_element_batch
from ztf_data import load_ztf_det_all, ztf_reindex, load_ztf_nearest_ast, make_ztf_near_elt, calc_hit_freq, load_ztf_easy_batch
from astro_utils import deg2dist

from typing import Dict

## Load Data for First 64 Asteroids

In [3]:
# Asteroids to load
n0 = 1
n1 = n0+64
# Manually specify date range
mjd0 = 58270.0
mjd1 = 58906.0
progbar=True

In [4]:
# Load data in this date range
df_ast_load, df_earth_load, df_sun_load = load_ast_data(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1, progbar=progbar)

Loading asteroid data from n0=1 to n1=65 in 1 blocks...


In [5]:
# df_ast_load

In [6]:
# Orbital elements for first 64 asteroids (dict of numpy arrays)
asteroid_nums = np.arange(n0, n1, dtype=np.int32)
elts = orbital_element_batch(asteroid_nums)

In [7]:
# Calculate vectors from these elements on the fly with calc_ast_data
df_ast_calc, df_earth_calc, df_sun_calc = calc_ast_data(elts=elts, mjd0=mjd0, mjd1=mjd1, element_id=asteroid_nums)

In [8]:
# df_ast_calc

In [9]:
# Check agreement
cols_q = ['qx', 'qy', 'qz']
cols_v = ['vx', 'vy', 'vz']

dq_ast = df_ast_calc[cols_q] - df_ast_load[cols_q]
dv_ast = df_ast_calc[cols_v] - df_ast_load[cols_v]

dq_earth = df_earth_calc[cols_q] - df_earth_load[cols_q]
dv_earth = df_earth_calc[cols_v] - df_earth_load[cols_v]

dq_sun = df_sun_calc[cols_q] - df_sun_load[cols_q]
dv_sun = df_sun_calc[cols_v] - df_sun_load[cols_v]

# Report
# print(f'mean position difference = {np.mean(np.linalg.norm(dq, axis=1)):6.2e}')
print('Mean position difference in AU:')
print(f'asteroids: {np.mean(np.linalg.norm(dq_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dq_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dq_sun, axis=1)):6.2e}')

print('\nMean velocity difference in AU/day:')
print(f'asteroids: {np.mean(np.linalg.norm(dv_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dv_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dv_sun, axis=1)):6.2e}')

Mean position difference in AU:
asteroids: 3.13e-07
earth    : 3.07e-12
sun      : 1.52e-15

Mean velocity difference in AU/day:
asteroids: 1.24e-09
earth    : 5.31e-14
sun      : 1.92e-18


## Splined Direction at ZTF Dates

In [10]:
# All the ZTF detections
ztf, mjd_unq = load_ztf_det_all()

In [11]:
# ztf

In [12]:
# mjd for the spline output
mjd = mjd_unq

In [13]:
# Spline the first 64 asteroids at the unique ZTF dates using the loaded asteroid data
df_ast_load_out, df_earth_load_out, df_sun_load_out = spline_ast_vec_df(df_ast=df_ast_load, df_earth=df_earth_load, df_sun=df_sun_load, mjd=mjd, include_elts=False)

In [14]:
# df_ast_load_out

In [15]:
# Spline the first 64 asteroids at the unique ZTF dates using the calculated asteroid data
df_ast_calc_out, df_earth_calc_out, df_sun_calc_out = spline_ast_vec_df(df_ast=df_ast_calc, df_earth=df_earth_calc, df_sun=df_sun_calc, mjd=mjd, include_elts=False)

In [16]:
# df_ast_calc_out

In [17]:
# Check agreement
cols_q = ['qx', 'qy', 'qz']
cols_v = ['vx', 'vy', 'vz']

dq_ast = df_ast_calc_out[cols_q] - df_ast_load_out[cols_q]
dv_ast = df_ast_calc_out[cols_v] - df_ast_load_out[cols_v]

dq_earth = df_earth_calc_out[cols_q] - df_earth_load_out[cols_q]
dv_earth = df_earth_calc_out[cols_v] - df_earth_load_out[cols_v]

dq_sun = df_sun_calc_out[cols_q] - df_sun_load_out[cols_q]
dv_sun = df_sun_calc_out[cols_v] - df_sun_load_out[cols_v]

# Report
# print(f'mean position difference = {np.mean(np.linalg.norm(dq, axis=1)):6.2e}')
print('Mean position difference in AU:')
print(f'asteroids: {np.mean(np.linalg.norm(dq_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dq_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dq_sun, axis=1)):6.2e}')

print('\nMean velocity difference in AU/day:')
print(f'asteroids: {np.mean(np.linalg.norm(dv_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dv_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dv_sun, axis=1)):6.2e}')

Mean position difference in AU:
asteroids: 3.08e-07
earth    : 5.02e-12
sun      : 2.48e-15

Mean velocity difference in AU/day:
asteroids: 1.22e-09
earth    : 8.67e-14
sun      : 3.13e-18


## Assemble Batch of Asteroids Near Orbital Elements

In [18]:
# Load nearest asteroid data
ztf_ast = load_ztf_nearest_ast()

In [19]:
# Asteroid numbers and hit counts
ast_num, hit_count = calc_hit_freq(ztf=ztf_ast, thresh_sec=2.0)

# Sort the hit counts in descending order and find the top 64
idx = np.argsort(hit_count)[::-1][0:64]

# Extract the asteroid number and hit count for this batch
ast_nums_best = ast_num[idx]
hit_count_best = hit_count[idx]

In [20]:
# Orbital elements for best asteroids (dict of numpy arrays)
element_id_best = np.sort(ast_nums_best)
elts_best = orbital_element_batch(element_id_best)

In [21]:
# Compute mjd0 and mjd1 from mjd_unq
mjd0 = np.floor(np.min(mjd_unq))
mjd1 = np.ceil(np.max(mjd_unq))

# Calculate positions in this date range, sampled daily
df_ast_daily, df_earth_daily, df_sun_daily = calc_ast_data(elts=elts_best, mjd0=mjd0, mjd1=mjd1, element_id=element_id_best)

In [22]:
# Spline positions at ztf times
mjd = mjd_unq
df_ast, df_earth, df_sun = spline_ast_vec_df(df_ast=df_ast_daily, df_earth=df_earth_daily, df_sun=df_sun_daily, mjd=mjd, include_elts=False)

In [23]:
# Direction from palomar
df_dir = calc_ast_dir(df_ast=df_ast, df_earth=df_earth, site_name='palomar')

In [24]:
# df_dir

In [25]:
# Calculate subset of ZTF data within threshold of this batch
thresh_deg = 1.0
probar = True
ztf_tbl = make_ztf_near_elt(ztf=ztf, df_dir=df_dir, thresh_deg=thresh_deg, progbar=progbar)

In [26]:
# Choose one of the ZTF data sets
element_test = ast_nums_best[0]
ztf_i = ztf_tbl[element_test]

In [27]:
ztf_i

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz
53851,b'ZTF18abnothj',594197584815010004,5501,58348.197581,266.229165,-13.513802,-0.063945,-0.983101,0.171530
73604,b'ZTF18ablwzmb',594197584815015003,5501,58348.197581,265.761024,-13.509148,-0.071871,-0.982578,0.171389
82343,b'ZTF18abiydvm',635193253015015018,12089,58389.193252,270.331454,-11.244934,0.005674,-0.977422,0.211222
257221,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,0.470042
327000,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,0.481893
...,...,...,...,...,...,...,...,...,...
5585430,b'ZTF20aaqjlxm',1149165065915015030,96202,58903.165069,58.775292,21.231723,0.483209,0.875373,0.015191
5585539,b'ZTF20aaqjkpu',1149165064115015000,96202,58903.165069,57.817755,20.738491,0.498104,0.867059,0.010035
5585636,b'ZTF20aaqjlxl',1149165065915015014,96202,58903.165069,58.639323,21.532043,0.484105,0.874763,0.020775
5585648,b'ZTF20aaqjkqe',1149165064115015002,96202,58903.165069,58.007009,20.618973,0.495877,0.868362,0.007343


## Test that Near Batch is a Superset of Observations with Matching Nearest Asteroid Number

In [28]:
# Assemble observations nearest to selected asteroid that are also within the threshold
thresh_dist = deg2dist(thresh_deg)
mask_test = (ztf_ast.nearest_ast_num == element_test) & (ztf_ast.nearest_ast_dist < thresh_dist)

# The test subset
ztf_test = ztf_ast[mask_test]

In [29]:
ztf_test

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob,nearest_ast_num,nearest_ast_dist,ast_ra,ast_dec,ast_ux,ast_uy,ast_uz
257221,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,0.470042,19.289200,0.810351,51921,0.008713,30.365848,42.217117,0.639004,0.610779,0.467571
327000,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,0.481893,17.725201,0.991371,51921,0.007949,33.147321,43.604733,0.606278,0.637608,0.475272
336873,b'ZTF19abizrbv',937428210615015010,40798,58691.428218,32.768773,44.192434,0.602900,0.633333,0.485181,19.924500,0.983333,51921,0.011358,33.130141,43.596052,0.606484,0.637450,0.475221
340306,b'ZTF19abiyxfi',937402265115010001,40777,58691.402269,33.223154,43.100462,0.610808,0.638835,0.467765,20.192900,0.841761,51921,0.008648,33.118298,43.590070,0.606626,0.637341,0.475186
341182,b'ZTF19abizrbu',937428210615015008,40798,58691.428218,33.030390,44.317294,0.599847,0.635715,0.485850,19.837000,0.985742,51921,0.012650,33.130141,43.596052,0.606484,0.637450,0.475221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5379997,b'ZTF20aaogcyq',1142172456115015014,94578,58896.172454,55.968493,21.904633,0.519245,0.853848,0.036427,17.839100,1.000000,51921,0.000007,55.968085,21.904751,0.519250,0.853845,0.036431
5450439,b'ZTF20aapdkvt',1145124016315015006,95210,58899.124016,57.062507,21.552337,0.505708,0.862296,0.026539,17.545500,1.000000,51921,0.000007,57.062108,21.552442,0.505713,0.862293,0.026542
5569973,b'ZTF20aaqifhg',1149110835915015022,96107,58903.110833,58.576908,21.111013,0.486363,0.873648,0.013797,18.046301,1.000000,51921,0.000007,58.576520,21.111107,0.486368,0.873645,0.013800
5576802,b'ZTF20aaqiwtw',1149142105915015021,96161,58903.142107,58.588750,21.107654,0.486209,0.873735,0.013700,18.070601,1.000000,51921,0.000007,58.588361,21.107762,0.486214,0.873732,0.013703


In [30]:
# These indices on the test subset should be contained in the subset of ztf close to the elements, ztf_i
index_hit = ztf_test.index

In [31]:
ztf_i.loc[index_hit]

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz
257221,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,0.470042
327000,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,0.481893
336873,b'ZTF19abizrbv',937428210615015010,40798,58691.428218,32.768773,44.192434,0.602900,0.633333,0.485181
340306,b'ZTF19abiyxfi',937402265115010001,40777,58691.402269,33.223154,43.100462,0.610808,0.638835,0.467765
341182,b'ZTF19abizrbu',937428210615015008,40798,58691.428218,33.030390,44.317294,0.599847,0.635715,0.485850
...,...,...,...,...,...,...,...,...,...
5379997,b'ZTF20aaogcyq',1142172456115015014,94578,58896.172454,55.968493,21.904633,0.519245,0.853848,0.036427
5450439,b'ZTF20aapdkvt',1145124016315015006,95210,58899.124016,57.062507,21.552337,0.505708,0.862296,0.026539
5569973,b'ZTF20aaqifhg',1149110835915015022,96107,58903.110833,58.576908,21.111013,0.486363,0.873648,0.013797
5576802,b'ZTF20aaqiwtw',1149142105915015021,96161,58903.142107,58.588750,21.107654,0.486209,0.873735,0.013700


## Load Easy Batch

In [35]:
ztf_batch = load_ztf_easy_batch(batch_size=64, thresh_deg=1.0)

In [33]:
ztf_batch

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz
37606,b'ZTF18abtdsvv',621336711115010010,9577,58375.336713,350.405941,0.879484,0.985897,-0.146790,0.080371
37607,b'ZTF18abtdsvv',621309991115010019,9572,58375.310000,350.405990,0.879483,0.985897,-0.146789,0.080371
37610,b'ZTF18abtdsvv',614358404915010002,8240,58368.358403,350.405972,0.879528,0.985897,-0.146789,0.080371
37612,b'ZTF18abtdsvv',615324371115015021,8425,58369.324375,350.405987,0.879540,0.985897,-0.146789,0.080371
141833,b'ZTF18abwpfho',623339405515010000,9980,58377.339410,348.764546,0.189321,0.980829,-0.177448,0.080534
...,...,...,...,...,...,...,...,...,...
5584064,b'ZTF18aaaknvz',1149162620015010022,96198,58903.162627,67.404504,45.304078,0.270241,0.878532,0.393893
5584155,b'ZTF20aaqjjhn',1149162620115015001,96198,58903.162627,66.164947,45.635778,0.282557,0.871180,0.401505
5584262,b'ZTF20aaqjltu',1149162621915015008,96198,58903.162627,67.072721,46.398450,0.268658,0.870797,0.411746
5584879,b'ZTF20aaqjltx',1149162621915015001,96198,58903.162627,66.993445,46.563928,0.268718,0.869478,0.414486
